## Method 1

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
ds = tfds.load('squad', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)

In [10]:
for i in ds:
    pass

## Method 2 (Used)

In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd

In [2]:
f = open('train-v2.0.json')
json_file = json.load(f)

In [ ]:
title = []
context = []
question = []
answer = []
for i in json_file['data']:
    title.append(i['title'])
    tempq = []
    tempa = []
    tempc = ''
    for j in i['paragraphs']:
        tempc += j['context'] + ' '
        for k in j['qas']:
            if len(k['answers']) == 0:
                continue
            tempq.append(k['question'])
            tempa.append(k['answers'][0])
    context.append(tempc)
    question.append(tempq)
    answer.append(tempa)

In [37]:
from nltk.tokenize import sent_tokenize

In [57]:
f = open('train-v2.0.json')
json_file = json.load(f)

In [81]:
qg_dataset = []
num_qa = 0
num_exact = 0 

for i in json_file['data']:
    title = i['title']
    context = []
    question = []
    answer = []
    for j in i['paragraphs']:
        for k in j['qas']:
            if len(k['answers']) == 0:
                continue
            
            num_qa+=1
            qg_row = {}

            qg_row['question'] = k['question']
            sentences = sent_tokenize(j['context'])
            found = False
            for ans in k['answers']:
                for s in range(len(sentences)):
                    if ans['text'] in sentences[s] and not found:
                        num_exact+=1
                        sent = sentences[s]
                        found = True
                        break
                    
            qg_row['sentence'] = sent
            qg_row['answer'] = ans['text']
            qg_row['context'] = j['context']
            qg_row['exact'] = found
            qg_dataset.append(qg_row)
            
print(f"Exact span found for {round(num_exact/num_qa, 3)} of data")

Exact span found for 0.998 of data


In [82]:
qg_df = pd.DataFrame(qg_dataset)

In [83]:
qg_df.iloc[0].to_dict()

{'question': 'When did Beyonce start becoming popular?',
 'sentence': "Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.",
 'answer': 'in the late 1990s',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby 

In [84]:
# failed to identify sentence containing answer
qg_df[qg_df['exact']==False]

,question,sentence,answer,context,exact
895,What was the first of Chopin's works to gain i...,The success of this concert led to an invitati...,Rondo Op. 1,From September 1823 to 1826 Chopin attended th...,False
900,What is the title of his first commercially su...,The success of this concert led to an invitati...,Rondo Op. 1.,From September 1823 to 1826 Chopin attended th...,False
928,Which of Chopin's works was unknowingly dedica...,He was also attracted to the singing student K...,Piano Concerto No. 1 (in E minor),Four boarders at his parents' apartments becam...,False
933,What musical piece had a Larghetto dedicated t...,Four boarders at his parents' apartments becam...,Piano Concerto No. 1 (in E minor),Four boarders at his parents' apartments becam...,False
938,What piece did Frédéric create specifically fo...,"On an 1829 return trip to Berlin, he was a gue...",Introduction and Polonaise brillante in C majo...,"In September 1828 Chopin, while still a studen...",False
...,...,...,...,...,...
85043,Which president did Notre Dame have in 1947?,"He wrote, ""Notre Dame football is a spiritual ...","Rev. John J. Cavanaugh, C.S.C.","The Rev. John J. Cavanaugh, C.S.C. served as p...",False
85073,Which person oversaw the creation of a science...,"The Grotto of Our Lady of Lourdes, which was b...",Fr. Zahm,A Science Hall was built in 1883 under the dir...,False
85249,"Who is responsible for writing the music for ""...","Michael J. Shea, a 1904 graduate, wrote the mu...",Rev. Michael J. Shea,"The ""Notre Dame Victory March"" is the fight so...",False
85259,Who is the current president of Notre Dame?,A notable alumnus of the College of Science is...,Rev. John Jenkins,Notre Dame alumni work in various fields. Alum...,False


In [87]:
qg_df[qg_df['exact']==True].to_csv('qg_train.csv', encoding='utf-8')

In [10]:
df = pd.DataFrame({'title':title, 'context':context, 'question':question, 'answer':answer})

In [15]:
df.to_pickle('data.df')

In [16]:
df = pd.read_pickle('data.df')

In [17]:
for i in range(len(df)):
    if len(df['question'][i])!=len(df['answer'][i]):
        print('lol')